# Data Dive 4 - Surfboard Specification and Review Data
## By: Alex, Cayson, and William
## Due: 4/13/21

# Objective: What is the most perfered Surfboard based on maneuverability and board type.

## DATA Source: https://www.kaggle.com/loureiro85/surfing

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import nltk

pd.set_option('display.max_columns',200) #allows for up to 500 columns to be displayed when viewing a dataframe
pd.set_option('display.max_rows',100)
plt.style.use('seaborn') # a style that can be used for plots - see style reference above

import warnings

import missingno as msn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import scipy.sparse as sp
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Importing our Data

In [3]:
df_surf = pd.read_csv("df_surf.csv")

In [4]:
df_surf.head(10)

,Unnamed: 0,board_adequate,board_how_many,board_length,board_nose_rocker,board_nose_shape,board_tail_rocker,board_tail_shape,board_thickness,board_type,board_volume,board_width,manoeuvres_01_paddling,manoeuvres_02_drop,manoeuvres_03_straight_ahead,manoeuvres_04_wall_riding,manoeuvres_05_floater,manoeuvres_06_cut_back,manoeuvres_07_carve,manoeuvres_08_off_the_lip,manoeuvres_09_tube,manoeuvres_10_air,performance_control,performance_ease_paddling,performance_flotation,performance_hold,performance_manoeuvrability,performance_passing_through,performance_stability,performance_surf_speed,surfer_age,surfer_exercise_frequency,surfer_experience,surfer_gender,surfer_height,surfer_style,surfer_weight,surfer_weight_distribution,wave_height,wave_shape,wave_height_min,wave_height_max,wave_height_mean,wave_height_interval
0,0,More or less,5.0,1.8034,4.75,Pointed,2.3125,"Outward (Round, Round-Pin, Pin)",2.3125,Shortboard,26.7,18.3125,Always,Always,Always,Always,Always,Usually,Usually,Usually,Usually,Sometimes,Good,Good,Good,Good,0.666667,Good,Good,Good,32.0,Once a week,Advanced,Male,1.72,Neutral,78.0,NaN,3,Spilling/Plunging,3.0,3.0,3.0,0.0
1,1,More or less,1.0,1.9304,NaN,Pointed,NaN,"Neutral (Squash, Square)",NaN,All-around,NaN,NaN,Usually,Sometimes,Not yet,Not yet,Not yet,Not yet,Not yet,Not yet,Not yet,Not yet,I dont know,Average,Excellent,I dont know,0.333333,Poor,Good,Poor,31.0,Twice a week,First-timer,Female,1.58,Smooth (recreative),54.0,NaN,2,Spilling,2.0,2.0,2.0,0.0
2,2,Very suitable,3.0,1.7272,NaN,Pointed,NaN,"Neutral (Squash, Square)",2.5625,Shortboard,30.5,20.3750,Always,Usually,Always,Usually,Sometimes,Usually,Usually,Usually,Sometimes,Not yet,Excellent,Average,Good,Excellent,1.000000,Good,Good,Good,29.0,Once a month,Advanced,Male,1.73,Neutral,75.0,NaN,"2, 5","Spilling, Spilling/Plunging, Plunging",2.0,5.0,3.5,3.0
3,3,Very suitable,1.0,1.8034,NaN,Pointed,NaN,"Neutral (Squash, Square)",2.4375,Shortboard,30.6,19.3750,Always,Usually,I dont know,Usually,Sometimes,Sometimes,Usually,Usually,Sometimes,Not yet,Good,Good,Excellent,Excellent,1.000000,Excellent,Good,Excellent,34.0,Twice a week,Advanced,Male,1.76,Neutral,76.0,NaN,"3, 6","Spilling, Spilling/Plunging, Plunging",3.0,6.0,4.5,3.0
4,4,Suitable,5.0,1.7780,NaN,Pointed,NaN,"Inward (Fish-tail, Swallow, Bat wing, Half moon)",NaN,All-around,NaN,NaN,Always,Usually,Usually,Always,Usually,Usually,Always,Always,Sometimes,Not yet,Good,Good,Average,Excellent,0.833333,Excellent,Excellent,Good,53.0,Twice a week,Advanced,Male,1.77,Neutral,84.0,NaN,5,Spilling/Plunging,5.0,5.0,5.0,0.0
5,5,Suitable,9.0,1.8288,NaN,Pointed,NaN,"Neutral (Squash, Square)",2.6250,Shortboard,32.5,19.5000,Always,Always,Always,Always,Always,Always,Always,Always,Always,Sometimes,Good,Good,Good,Good,0.833333,Good,Good,Good,42.0,Twice a week,Advanced,Male,1.88,Agressive (competitive),90.0,NaN,5,Spilling/Plunging,5.0,5.0,5.0,0.0
6,6,More or less,2.0,1.8034,NaN,Pointed,NaN,"Outward (Round, Round-Pin, Pin)",NaN,All-around,31.2,NaN,Usually,Usually,Always,Always,Sometimes,Sometimes,Not yet,Sometimes,Not yet,I dont know,Average,Average,Good,Average,0.833333,Excellent,Good,Average,34.0,Twice a week,Intermediate,Male,1.74,Smooth (recreative),81.0,NaN,"3, 4","Spilling, Spilling/Plunging",3.0,4.0,3.5,1.0
7,7,Very suitable,3.0,NaN,NaN,Pointed,NaN,"Inward (Fish-tail, Swallow, Bat wing, Half moon)",NaN,Shortboard,NaN,NaN,Always,Always,Always,Always,Always,Always,Always,Always,Usually,Sometimes,Excellent,Good,Good,Excellent,1.000000,Excellent,Excellent,Excellent,28.0,Five times a week,Advanced,Male,1.75,Agressive (competitive),70.0,NaN,"3, 4","Spilling, Spilling/Plunging",3.0,4.0,3.5,1.0
8,8,Suitable,3.0,1.7780,NaN,Pointed,NaN,"Neutral (Squash, Square)",2.2500,Shortboard,26.8,18.6250,Always,Always,Always,Always,Always,NaN,Always,Always,Always,Sometimes,Good,Average,Average,Good,0.833333,Good,Good,Average,31.0,Twice a week,Advanced,Male,1.76,Agressive (competitive),73.0,NaN,5,Spilling/Plunging,5.0,5.0,5.0,0.0
9,9,Suitable,2.0,1.8034,NaN,Pointe

In [6]:
df_surf.columns

Index(['Unnamed: 0', 'board_adequate', 'board_how_many', 'board_length',
       'board_nose_rocker', 'board_nose_shape', 'board_tail_rocker',
       'board_tail_shape', 'board_thickness', 'board_type', 'board_volume',
       'board_width', 'manoeuvres_01_paddling', 'manoeuvres_02_drop',
       'manoeuvres_03_straight_ahead', 'manoeuvres_04_wall_riding',
       'manoeuvres_05_floater', 'manoeuvres_06_cut_back',
       'manoeuvres_07_carve', 'manoeuvres_08_off_the_lip',
       'manoeuvres_09_tube', 'manoeuvres_10_air', 'performance_control',
       'performance_ease_paddling', 'performance_flotation',
       'performance_hold', 'performance_manoeuvrability',
       'performance_passing_through', 'performance_stability',
       'performance_surf_speed', 'surfer_age', 'surfer_exercise_frequency',
       'surfer_experience', 'surfer_gender', 'surfer_height', 'surfer_style',
       'surfer_weight', 'surfer_weight_distribution', 'wave_height',
       'wave_shape', 'wave_height_min', 'wave_hei

## Data Dictionary

### board_adequate - Board Adaquacy

### board_how_many - How many of that surfboard used

### board_length - Length of board

### board_nose_rocker - Nose rocker angle (This is what prevents the board from cutting into a wave)
       
### board_nose_shape - Shape of nose
           
### board_tail_rocker - Tail rocker angle
      
### board_tail_shape - Shape of tail

### board_thickness - Thickness of the surfboard

### board_type - Type of board used

### board_volume - Volume of the board
      
### board_width - Width of the board

### manoeuvres_01_paddling - Laying on Stomach paddling to gain momentum

### manoeuvres_02_drop - Being pulled into a large wave (Usually using a jetski) 

### manoeuvres_03_straight_ahead - Surfing straight into a wave

### manoeuvres_04_wall_riding - Riding the inner wall of a wave

### manoeuvres_05_floater - Gliding over the broken tops of a wave

### manoeuvres_06_cut_back - Turn that takes the surfer back to the curl of the wave

### manoeuvres_07_carve - A sharp turn used to aggressively turn into a wave

### manoeuvres_08_off_the_lip - The surfer rides to the top of a wave then cuts down the inside without losing momentum
       
### manoeuvres_09_tube - Riding the the inside of a fully curled wave

### manoeuvres_10_air - Quickly riding up a wave to jump over it

### performance_control - How well the board controls

### performance_ease_paddling - How eacy it is to paddle on the board

### performance_flotation - How well the board floats

### performance_hold - How strong the hold is on the board

### performance_manoeuvrability - How easy is it to perform manoeuvres on the board. 

### performance_passing_through - How well does the board pass through waves

### performance_stability - How stable is the board

### performance_surf_speed - How fast is the board

### surfer_age - How old is the surfer

### surfer_exercise_frequency - How often does the surfer exercise 

### surfer_experience - How much experience doe shte surfer have

### surfer_gender - What gender is the surfer

### surfer_height - What is the surfers Height

### surfer_style- What style does the surfer use

### surfer_weight - What is the surfers weight

### surfer_weight_distribution - Board weight distribution 

### wave_height - height of the wave

### wave_shape - Shape of the wave

### wave_height_min - Minimum wave height while using the board

### wave_height_max - Maximum wave height while using the board

### wave_height_mean - Average wave height while using the board

### wave_height_interval - Distance between waves

# Part 1 Data Cleansing and Feature Engieneering 

In [7]:
## Data Cleansing... Many Null variables appear to center around amatuer surfers.
## Potentially explore asking perfered boards of expert surfers. 

## Create DataFrames for NLP Sentiement Analysis

# Part 2 Visualization 

In [8]:
## Potential for analysing relationships between variables regarding Manoeuvres and board information and Surfer Experience. 


# Part 3 Natural Language Processing - Sentiement Analysis